In [16]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [17]:
dataset = 'JAW2011'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/pone.0024918.s001.csv', header=3)
print('Raw data loaded.')
data

Loading raw data for JAW2011 ...
Raw data loaded.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_10885/2010846759.py:4: DtypeWarning: Columns (201) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/pone.0024918.s001.csv', header=3)


,id,Protein Group IDs,Positions,Position,Known Site,Peptide IDs,Mod. Peptide IDs,Best Localization Evidence ID,Best Localization MS/MS ID,Best Localization Raw File,...,Intensity L LS1b,Intensity H LS1b,Intensity LS2,Intensity L LS2,Intensity H LS2,Intensity HS,Intensity L HS,Intensity H HS,Reverse,Contaminant
0,4007,10,277,277,NaN,9051,10058,35733,83586,o101004_RD463HCLSb_i53,...,0,0,4713700,2066900,2646800,0,0,0,NaN,NaN
1,5025,10,281,281,NaN,9051,10057,35732,83585,o101004_RD463HCLSb_i53,...,0,0,4713700,2066900,2646800,0,0,0,NaN,NaN
2,10,11,456,456,NaN,7883,8756,31086,72514,o100702_RD463HSLCi_55,...,0,0,1101900,296780,805110,0,0,0,NaN,NaN
3,11,11,458,458,Phosphoserine,7883,8756,31087,72515,o101004_RD463HCLSb_i53,...,0,0,2313200,641920,1671200,0,0,0,NaN,NaN
4,12,12,573,573,Phosphoserine,7086,7828,27993,65155,o101004_RD463HCLSb_i45,...,15975000,20560000,15075000,8638100,6436500,19202000,7302000,11900000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5008,4544,1546,155,155,NaN,3560,3911,13999,33001,o100702_RD463HSLCi_35,...,0,0,0,0,0,889940,465110,424830,NaN,NaN
5009,3366,2432,1065,1065,NaN,28;6907,30;7631,74,148,o101004_RD463HCLSb_i51,...,0,0,803650,185960,617690,0,0,0,NaN,NaN
5010,3367,2432,1066,1066,NaN,28;6907,30;7631,27318,63557,o101004_RD463HCLSb_i53,...,0,0,1500000,400220,1099800,0,0,0,NaN,NaN
5011,2302,1777,477,477,NaN,8260,9183,32571,76052,o100702_RD463HSLCi_33,...,277510,553710,2950100,1232200,1717900,8270400,3437500,4832800,NaN,NaN


In [18]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization Prob'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino Acid'].str.contains(';', na=False)]
data = data[~data['Position'].astype(str).str.contains(';', na=False)]
data = data[~data['Gene Names'].str.contains(';', na=False)]
data

,id,Protein Group IDs,Positions,Position,Known Site,Peptide IDs,Mod. Peptide IDs,Best Localization Evidence ID,Best Localization MS/MS ID,Best Localization Raw File,...,Intensity L LS1b,Intensity H LS1b,Intensity LS2,Intensity L LS2,Intensity H LS2,Intensity HS,Intensity L HS,Intensity H HS,Reverse,Contaminant
37,59,48,275,275,Phosphoserine,8716,9694,34365,80295,o100806_RD463HCLSa_37i,...,96350,183180,0,0,0,0,0,0,NaN,NaN
39,61,49,137,137,Phosphoserine,7694,8539,30398,70822,o100802_RD463HCLSa_43i,...,100160,486010,1417200,624230,792990,0,0,0,NaN,NaN
40,62,51,405,405,NaN,6606,7309,26386,61397,o101004_RD463HCLSb_i47,...,0,0,912350,383320,529030,0,0,0,NaN,NaN
47,4027,59,195,195,Phosphothreonine,3607,3967,14218,33536,o100802_RD463HCLSa_49i,...,0,0,0,0,0,2198700,1366800,831980,NaN,NaN
69,89,66,517,517,Phosphoserine,1649,1808,6663,15664,o101004_RD463HCLSb_i39,...,0,0,1398600,645300,753270,1628400,872650,755750,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4981,4664,1835,369,369,Phosphothreonine,5271,5788,21294,50177,o100806_RD463HCLSa_37i,...,953440,814900,711330,497010,214310,652990,388020,264970,NaN,NaN
4982,4665,1835,388,388,NaN,8713,9691,34355,80274,o100802_RD463HCLSa_41i,...,1531400,797280,0,0,0,0,0,0,NaN,NaN
4987,3963,2748,72,72,NaN,9306,10335,36796,86061,o100802_RD463HCLSa_47i,...,0,0,0,0,0,5750400,2095800,3654500,NaN,NaN
4988,3964,2748,75,75,NaN,9306,10335,36795,86060,o100802_RD463HCLSa_47i,...,0,0,0,0,0,0,0,0,NaN,NaN


In [19]:
# filter data
data['Sequence Window'] = data['Sequence Window'].str.replace('_', '')
data

,id,Protein Group IDs,Positions,Position,Known Site,Peptide IDs,Mod. Peptide IDs,Best Localization Evidence ID,Best Localization MS/MS ID,Best Localization Raw File,...,Intensity L LS1b,Intensity H LS1b,Intensity LS2,Intensity L LS2,Intensity H LS2,Intensity HS,Intensity L HS,Intensity H HS,Reverse,Contaminant
37,59,48,275,275,Phosphoserine,8716,9694,34365,80295,o100806_RD463HCLSa_37i,...,96350,183180,0,0,0,0,0,0,NaN,NaN
39,61,49,137,137,Phosphoserine,7694,8539,30398,70822,o100802_RD463HCLSa_43i,...,100160,486010,1417200,624230,792990,0,0,0,NaN,NaN
40,62,51,405,405,NaN,6606,7309,26386,61397,o101004_RD463HCLSb_i47,...,0,0,912350,383320,529030,0,0,0,NaN,NaN
47,4027,59,195,195,Phosphothreonine,3607,3967,14218,33536,o100802_RD463HCLSa_49i,...,0,0,0,0,0,2198700,1366800,831980,NaN,NaN
69,89,66,517,517,Phosphoserine,1649,1808,6663,15664,o101004_RD463HCLSb_i39,...,0,0,1398600,645300,753270,1628400,872650,755750,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4981,4664,1835,369,369,Phosphothreonine,5271,5788,21294,50177,o100806_RD463HCLSa_37i,...,953440,814900,711330,497010,214310,652990,388020,264970,NaN,NaN
4982,4665,1835,388,388,NaN,8713,9691,34355,80274,o100802_RD463HCLSa_41i,...,1531400,797280,0,0,0,0,0,0,NaN,NaN
4987,3963,2748,72,72,NaN,9306,10335,36796,86061,o100802_RD463HCLSa_47i,...,0,0,0,0,0,5750400,2095800,3654500,NaN,NaN
4988,3964,2748,75,75,NaN,9306,10335,36795,86060,o100802_RD463HCLSa_47i,...,0,0,0,0,0,0,0,0,NaN,NaN


In [20]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 

In [21]:
data = match_seq_to_genename(data, 'Sequence Window')

WLSPRVSPPASPG
Match found for sequence: ID: sp|Q9Y4U1|MMAC_HUMAN
Name: sp|Q9Y4U1|MMAC_HUMAN
Description: sp|Q9Y4U1|MMAC_HUMAN Cyanocobalamin reductase / alkylcobalamin dealkylase OS=Homo sapiens OX=9606 GN=MMACHC PE=1 SV=3
Number of features: 0
Seq('MEPKVAELKQKIEDTLCPFGFEVYPFQVAWYNELLPPAFHLPLPGPTLAFLVLS...PGP')
Gene name match: <re.Match object; span=(98, 107), match='GN=MMACHC'>
Match found: WLSPRVSPPASPG -> MMACHC
IIQRSFSSPENFQ
Match found for sequence: ID: sp|Q9Y580|RBM7_HUMAN
Name: sp|Q9Y580|RBM7_HUMAN
Description: sp|Q9Y580|RBM7_HUMAN RNA-binding protein 7 OS=Homo sapiens OX=9606 GN=RBM7 PE=1 SV=1
Number of features: 0
Seq('MGAAAAEADRTLFVGNLETKVTEELLFELFHQAGPVIKVKIPKDKDGKPKQFAF...SRH')
Gene name match: <re.Match object; span=(67, 74), match='GN=RBM7'>
Match found: IIQRSFSSPENFQ -> RBM7
NFYTLVSEGALDR
Match found for sequence: ID: sp|Q9Y5E6|PCDB3_HUMAN
Name: sp|Q9Y5E6|PCDB3_HUMAN
Description: sp|Q9Y5E6|PCDB3_HUMAN Protocadherin beta-3 OS=Homo sapiens OX=9606 GN=PCDHB3 PE=1 SV=2
Numb

In [22]:
data['Phosphosite'] = data['Amino Acid'].astype(str) + '(' + data['Position'].astype(str) + ')'


In [23]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite', 'GeneName'] + [
    col for col in data.columns
    if col.startswith('Ratio H/L') and
       'Normalized' not in col and
       'Significance' not in col and
       'Protein' not in col and
       'Localized' not in col and
       'Unmod' not in col and
       'Nmods' not in col and
       'Variability' not in col and
       'Occupancy' not in col and
       'Count' not in col
]
data = data[keepcols]
data

,Phosphosite,GeneName,Ratio H/L,Ratio H/L_1,Ratio H/L_2,Ratio H/L_3,Ratio H/L LS1a,Ratio H/L LS1a_1,Ratio H/L LS1a_2,Ratio H/L LS1a_3,...,Ratio H/L LS1b_2,Ratio H/L LS1b_3,Ratio H/L LS2,Ratio H/L LS2_1,Ratio H/L LS2_2,Ratio H/L LS2_3,Ratio H/L HS,Ratio H/L HS_1,Ratio H/L HS_2,Ratio H/L HS_3
37,S(275),MMACHC,0.50173,0.50173,NaN,NaN,0.55030,0.55030,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,S(137),RBM7,0.50637,0.50637,NaN,NaN,0.31853,0.31853,NaN,NaN,...,NaN,NaN,0.70986,0.70986,NaN,NaN,NaN,NaN,NaN,NaN
40,S(405),PCDHB3,0.74967,0.74967,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.74967,0.74967,NaN,NaN,NaN,NaN,NaN,NaN
47,T(195),ORC6,0.65950,0.65950,NaN,NaN,0.81763,0.81763,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.53195,0.53195,NaN,NaN
69,S(517),NUP88,0.76125,0.76125,NaN,NaN,0.71608,0.71608,NaN,NaN,...,NaN,NaN,0.87735,0.87735,NaN,NaN,0.76125,0.76125,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4981,T(369),NUP153,1.57000,1.57000,NaN,NaN,1.57850,1.57850,NaN,NaN,...,NaN,NaN,2.99420,2.99420,NaN,NaN,0.95234,0.95234,NaN,NaN
4982,T(388),NUP153,1.96480,1.96480,NaN,NaN,1.99960,1.99960,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4987,S(72),NaN,1.10330,1.10330,NaN,NaN,0.78598,0.78598,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.54880,1.54880,NaN,NaN
4988,S(75),VIPAS39,0.74450,0.74450,NaN,NaN,0.74450,0.74450,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# log2 transform the ratio columns (Ratio columns)
Ratio_columns = [
    col for col in data.columns
    if col.startswith('Ratio H/L') and
       'Normalized' not in col and
       'Significance' not in col and
       'Protein' not in col and
       'Localized' not in col and
       'Unmod' not in col and
       'Nmods' not in col and
       'Variability' not in col and
       'Occupancy' not in col and
       'Count' not in col
]
data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_10885/684745179.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')


In [25]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Ratio_columns]
    dataset[Ratio_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset


In [26]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite GeneName  Ratio H/L  Ratio H/L_1  Ratio H/L_2  Ratio H/L_3  \
37        S(275)   MMACHC  -0.995017    -0.995017          NaN          NaN   
39        S(137)     RBM7  -0.981736    -0.981736          NaN          NaN   
40        S(405)   PCDHB3  -0.415672    -0.415672          NaN          NaN   
47        T(195)     ORC6  -0.600555    -0.600555          NaN          NaN   
69        S(517)    NUP88  -0.393558    -0.393558          NaN          NaN   
...          ...      ...        ...          ...          ...          ...   
4981      T(369)   NUP153   0.650765     0.650765          NaN          NaN   
4982      T(388)   NUP153   0.974382     0.974382          NaN          NaN   
4987       S(72)      NaN   0.141825     0.141825          NaN          NaN   
4988       S(75)  VIPAS39  -0.425656    -0.425656          NaN          NaN   
4989      S(159)    CDK16   0.288418     0.288418          NaN

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_10885/1555182160.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Ratio_columns] = cols_to_transform.apply(np.log2)


,Phosphosite,GeneName,Ratio H/L,Ratio H/L_1,Ratio H/L_2,Ratio H/L_3,Ratio H/L LS1a,Ratio H/L LS1a_1,Ratio H/L LS1a_2,Ratio H/L LS1a_3,...,Ratio H/L LS1b_2,Ratio H/L LS1b_3,Ratio H/L LS2,Ratio H/L LS2_1,Ratio H/L LS2_2,Ratio H/L LS2_3,Ratio H/L HS,Ratio H/L HS_1,Ratio H/L HS_2,Ratio H/L HS_3
37,S(275),MMACHC,-0.995017,-0.995017,NaN,NaN,-0.861710,-0.861710,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,S(137),RBM7,-0.981736,-0.981736,NaN,NaN,-1.650499,-1.650499,NaN,NaN,...,NaN,NaN,-0.494394,-0.494394,NaN,NaN,NaN,NaN,NaN,NaN
40,S(405),PCDHB3,-0.415672,-0.415672,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.415672,-0.415672,NaN,NaN,NaN,NaN,NaN,NaN
47,T(195),ORC6,-0.600555,-0.600555,NaN,NaN,-0.290480,-0.290480,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.910637,-0.910637,NaN,NaN
69,S(517),NUP88,-0.393558,-0.393558,NaN,NaN,-0.481807,-0.481807,NaN,NaN,...,NaN,NaN,-0.188776,-0.188776,NaN,NaN,-0.393558,-0.393558,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4981,T(369),NUP153,0.650765,0.650765,NaN,NaN,0.658554,0.658554,NaN,NaN,...,NaN,NaN,1.582171,1.582171,NaN,NaN,-0.070451,-0.070451,NaN,NaN
4982,T(388),NUP153,0.974382,0.974382,NaN,NaN,0.999711,0.999711,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4987,S(72),NaN,0.141825,0.141825,NaN,NaN,-0.347435,-0.347435,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.631151,0.631151,NaN,NaN
4988,S(75),VIPAS39,-0.425656,-0.425656,NaN,NaN,-0.425656,-0.425656,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_10885/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [28]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data


In [29]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

There are no phosphosites with multiple measurements
After cleaning phosphosite_ID column:


,phosphosite_ID,Ratio H/L,Ratio H/L_1,Ratio H/L_2,Ratio H/L_3,Ratio H/L LS1a,Ratio H/L LS1a_1,Ratio H/L LS1a_2,Ratio H/L LS1a_3,Ratio H/L LS1b,...,Ratio H/L LS1b_2,Ratio H/L LS1b_3,Ratio H/L LS2,Ratio H/L LS2_1,Ratio H/L LS2_2,Ratio H/L LS2_3,Ratio H/L HS,Ratio H/L HS_1,Ratio H/L HS_2,Ratio H/L HS_3
37,MMACHC_S(275),-0.995017,-0.995017,NaN,NaN,-0.861710,-0.861710,NaN,NaN,-1.128314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,RBM7_S(137),-0.981736,-0.981736,NaN,NaN,-1.650499,-1.650499,NaN,NaN,-1.407364,...,NaN,NaN,-0.494394,-0.494394,NaN,NaN,NaN,NaN,NaN,NaN
40,PCDHB3_S(405),-0.415672,-0.415672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.415672,-0.415672,NaN,NaN,NaN,NaN,NaN,NaN
47,ORC6_T(195),-0.600555,-0.600555,NaN,NaN,-0.290480,-0.290480,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.910637,-0.910637,NaN,NaN
69,NUP88_S(517),-0.393558,-0.393558,NaN,NaN,-0.481807,-0.481807,NaN,NaN,NaN,...,NaN,NaN,-0.188776,-0.188776,NaN,NaN,-0.393558,-0.393558,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,NUP153_S(547),-0.162428,-0.162428,NaN,NaN,-0.305129,-0.305129,NaN,NaN,NaN,...,NaN,NaN,-0.019726,-0.019726,NaN,NaN,NaN,NaN,NaN,NaN
4981,NUP153_T(369),0.650765,0.650765,NaN,NaN,0.658554,0.658554,NaN,NaN,0.642933,...,NaN,NaN,1.582171,1.582171,NaN,NaN,-0.070451,-0.070451,NaN,NaN
4982,NUP153_T(388),0.974382,0.974382,NaN,NaN,0.999711,0.999711,NaN,NaN,0.948975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4988,VIPAS39_S(75),-0.425656,-0.425656,NaN,NaN,-0.425656,-0.425656,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/JAW2011.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

JAW2011 has been saved to CSV successfully!      phosphosite_ID  Ratio H/L  Ratio H/L_1  Ratio H/L_2  Ratio H/L_3  \
37    MMACHC_S(275)  -0.995017    -0.995017          NaN          NaN   
39      RBM7_S(137)  -0.981736    -0.981736          NaN          NaN   
40    PCDHB3_S(405)  -0.415672    -0.415672          NaN          NaN   
47      ORC6_T(195)  -0.600555    -0.600555          NaN          NaN   
69     NUP88_S(517)  -0.393558    -0.393558          NaN          NaN   
...             ...        ...          ...          ...          ...   
4979  NUP153_S(547)  -0.162428    -0.162428          NaN          NaN   
4981  NUP153_T(369)   0.650765     0.650765          NaN          NaN   
4982  NUP153_T(388)   0.974382     0.974382          NaN          NaN   
4988  VIPAS39_S(75)  -0.425656    -0.425656          NaN          NaN   
4989   CDK16_S(159)   0.288418     0.288418          NaN          NaN   

      Ratio H/L LS1a  Ratio H/L LS1a_1  Ratio H/L LS1a_2  Ratio H/L LS1a_3  \
3